In [1]:
import torch
import torchvision

# --- 1. MODEL PILOT (Řízení - Regrese) ---
# Tento model bude určovat směr jízdy (X, Y souřadnice)
model_pilot = torchvision.models.resnet18(pretrained=False)
model_pilot.fc = torch.nn.Linear(512, 2)  # Výstup: 2 hodnoty (x, y)

# --- 2. MODEL NAVIGÁTOR (Rozhodování - Klasifikace) ---
# Tento model bude poznávat situace (Křižovatka, Slepá, Cíl...)
model_navigator = torchvision.models.resnet18(pretrained=False)
model_navigator.fc = torch.nn.Linear(512, 3)  # Výstup: 3 tříd (dead end, goal, other)

In [2]:
# Načtení vah pro Pilota (model, který řídí zatáčení - Regrese)
# Tento soubor vznikne z trénování na Datasetu B (kde klikáte myší)
model_pilot.load_state_dict(torch.load('best_steering_model_xy.pth'))

# Načtení vah pro Navigátora (model, který pozná slepou uličku - Klasifikace)
# Tento soubor vznikne z trénování na Datasetu A (tlačítka)

model_navigator.load_state_dict(torch.load('best_model_maze.pth'))

<All keys matched successfully>

In [3]:
device = torch.device('cuda')

# 1. Pilot (Řízení) na GPU + optimalizace
model_pilot = model_pilot.to(device)
model_pilot = model_pilot.eval().half()

# 2. Navigátor (Rozhodování) na GPU + optimalizace
model_navigator = model_navigator.to(device)
model_navigator = model_navigator.eval().half()

In [4]:
import cv2
import numpy as np

# Definujeme střední hodnotu a odchylku rovnou jako Tensory na GPU v poloviční přesnosti (FP16)
# To výrazně zrychlí výpočet, protože nemusíme data kopírovat tam a zpět
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(camera_value):
    global device, mean, std
    x = camera_value
    
    # 1. Konverze z BGR (OpenCV) do RGB
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    
    # 2. Přehození os z HWC (Height, Width, Channel) na CHW (Channel, Height, Width)
    x = x.transpose((2, 0, 1))
    
    # 3. Převod z Numpy pole na Torch Tensor a přesun na GPU
    x = torch.from_numpy(x).to(device)
    
    # 4. Klíčový krok: Převod na FP16 (.half) a škálování z 0-255 na 0-1
    x = x.half().div(255.0)
    
    # 5. Normalizace (odečtení průměru a podělení odchylkou) přímo na GPU
    x = x.sub_(mean[:, None, None]).div_(std[:, None, None])
    
    # 6. Přidání batch dimenze (tvar [1, 3, 224, 224])
    x = x[None, ...]
    
    return x

In [5]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg, Robot

# 1. Inicializace Kamery
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)

# 2. Inicializace Robota (Přidáno)
robot = Robot()

# 3. Propojení kamery s widgetem (aby byl vidět obraz)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

# Zobrazení obrazu a tlačítka vedle sebe
display(widgets.HBox([image]))

In [6]:
import glob
import os
import time
import numpy as np
from PIL import Image, ImageFont, ImageDraw

# Cesta k fontům (necháváme stejné)
try:
    font = ImageFont.truetype("/usr/share/fonts/dejavu/DejaVuSans.ttf", 20)
    font_small = ImageFont.truetype("/usr/share/fonts/dejavu/DejaVuSans.ttf", 12)
except:
    font = ImageFont.load_default()
    font_small = ImageFont.load_default()

path_to_img_folder = 'images'

# Vytvoření složky
try:
    os.makedirs(path_to_img_folder)
except FileExistsError:
    print('Directory already exists')

# Definice názvů tříd (MUSÍ odpovídat abecednímu pořadí složek ve vašem datasetu!)
# Ověřte si pořadí pomocí dataset.class_to_idx v trénovacím notebooku
MAZE_CLASSES = ['dead_end', 'goal', 'other']

def save_frames_with_telemetry(raw_image, xy_pilot, nav_probs, action_str, is_recovering):
    """
    raw_image: Obrázek z kamery (numpy array)
    xy_pilot: [x, y] souřadnice z modelu řízení
    nav_probs: Seznam pravděpodobností z modelu navigace
    action_str: Textový popis akce (např. "DRIVING", "RECOVERING")
    is_recovering: Boolean, zda robot zrovna řeší slepou uličku
    """
    
    # Unikátní název souboru podle času
    img_file_nm = os.path.join(path_to_img_folder, f'img_{int(time.time() * 100)}.jpg')
    
    # Převod na PIL obrázek pro kreslení
    img = Image.fromarray(raw_image)
    draw = ImageDraw.Draw(img)
    width, height = img.size
    
    # --- 1. VIZUALIZACE PILOTA (Zelená tečka) ---
    # xy_pilot obsahuje hodnoty cca od -1 do 1. Převedeme na pixely.
    # X: -1 (vlevo) -> 0, 1 (vpravo) -> width
    # Y: -1 (dole) -> height, 1 (nahoře) -> 0 (JetBot má Y často invertované, záleží na tréninku)
    
    x_norm = (xy_pilot[0] + 1) / 2.0
    y_norm = (xy_pilot[1] + 1) / 2.0
    
    x_px = int(x_norm * width)
    y_px = int(y_norm * height)
    
    # Nakreslíme zelené kolečko tam, kam model "chce" jet
    r = 8
    draw.ellipse((x_px - r, y_px - r, x_px + r, y_px + r), fill=(0, 255, 0), outline='black')

    # --- 2. VIZUALIZACE NAVIGÁTORA (Text) ---
    # Zjistíme, která třída má největší pravděpodobnost
    max_idx = np.argmax(nav_probs)
    class_name = MAZE_CLASSES[max_idx] if max_idx < len(MAZE_CLASSES) else "Unknown"
    confidence = nav_probs[max_idx]
    
    # Vypsání textu
    telemetry_text = f"NAV: {class_name} ({confidence:.2f})"
    draw.text((10, 10), telemetry_text, fill=(255, 255, 0), font=font)
    
    # Výpis akce (řízení)
    action_text = f"ACT: {action_str}"
    draw.text((10, 35), action_text, fill=(255, 100, 100), font=font)
    
    # Zobrazení hodnoty zatáčení (X)
    steer_text = f"Steer X: {xy_pilot[0]:.2f}"
    draw.text((10, 60), steer_text, fill=(0, 255, 0), font=font_small)

    # --- 3. VAROVÁNÍ (Dead End / Recovering) ---
    if is_recovering:
        # Červený rámeček kolem celého obrazu
        draw.rectangle((0, 0, width-1, height-1), outline="red", width=5)
        draw.text((width//2 - 60, height//2), "RECOVERING", fill="red", font=font)
    elif class_name == 'dead_end' and confidence > 0.8:
         draw.text((width - 120, 10), "DEAD END!", fill="red", font=font)

    # Uložení
    img.save(img_file_nm)

In [24]:
import torch.nn.functional as F
import time
import numpy as np

# --- 1. NASTAVENÍ JÍZDY ---
# Rychlost: Pokud robot na 0.09 cuká a nejede plynule, zkuste 0.10
BASE_SPEED = 0.09           

# Citlivost: Zvedáme z 0.06 na 0.15. 
# Při 0.06 robot ignoruje malé chyby. Při 0.15 bude aktivněji korigovat směr.
STEERING_GAIN = 0.15        

# Kompenzace motorů:
# Pokud při jízdě rovně (X=0.00) robot uhýbá doprava, dejte sem záporné číslo (např. -0.05).
STEERING_BIAS = -0.02       

# --- LIMITY ---
MAX_STEERING_ANGLE = 0.30   
KD = 0.08                   

# --- LOGIKA ---
DEAD_END_THRESHOLD = 0.85   
GOAL_THRESHOLD = 0.85       
TIME_REVERSE = 0.6          
TIME_SPIN = 0.9             

# --- PROMĚNNÉ ---
angle = 0.0
angle_last = 0.0
is_recovering = False       
frame_counter = 0
skip_frames = 0  # NOVÉ: Počítadlo pro zahození starých snímků
data_log = [] 
last_time = time.time()

def execute(change):
    global angle, angle_last, is_recovering, frame_counter, robot, data_log, last_time, skip_frames
    
    # 1. ČIŠTĚNÍ BUFFERU (Řešení problému se slepou uličkou)
    # Pokud máme nastaveno přeskakování, ignorujeme tento snímek a ukončíme funkci.
    if skip_frames > 0:
        skip_frames -= 1
        return

    # Měření FPS
    curr_time = time.time()
    dt = curr_time - last_time
    fps = 1.0 / dt if dt > 0 else 0
    last_time = curr_time
    
    image = change['new']
    if is_recovering: return

    image_input = preprocess(image)

    # 2. NAVIGACE
    nav_output = model_navigator(image_input)
    nav_probs = F.softmax(nav_output, dim=1).flatten()
    prob_dead_end = float(nav_probs[0])
    prob_goal = float(nav_probs[1]) if len(nav_probs) > 2 else 0.0
    
    action_description = "DRIVING" 

    # --- Slepá ulice ---
    if prob_dead_end > DEAD_END_THRESHOLD:
        robot.stop() 
        is_recovering = True
        print(f"Slepá ({prob_dead_end:.2f}) -> Manévr")
        
        # Manévr
        time.sleep(0.2)
        robot.backward(0.15)
        time.sleep(TIME_REVERSE)
        robot.left(0.15) # Pokud točí málo, zvyšte na 0.30
        time.sleep(TIME_SPIN)
        robot.stop()
        
        # DŮLEŽITÉ: Nastavíme skip_frames na 20.
        # To znamená, že příštích 20 snímků (které jsou staré a ukazují zeď) zahodíme.
        skip_frames = 20 
        
        is_recovering = False
        angle = 0.0
        return

    # --- Cíl ---
    if prob_goal > GOAL_THRESHOLD:
        robot.stop()
        is_recovering = True 
        print("CÍL!")
        return

    # 3. ŘÍZENÍ (PILOT)
    xy_output = model_pilot(image_input)
    xy = xy_output.detach().float().cpu().numpy().flatten()
    raw_x = xy[0]
    
    # Normalizace
    x = (raw_x - 112.0) / 112.0
    
    # PID Výpočet
    angle = (x * STEERING_GAIN) + ((x - angle_last) * KD) + STEERING_BIAS
    angle_last = x
    
    angle = np.clip(angle, -MAX_STEERING_ANGLE, MAX_STEERING_ANGLE)
    
    # Motory
    left_value = max(0.0, min(1.0, BASE_SPEED + angle))
    right_value = max(0.0, min(1.0, BASE_SPEED - angle))
    
    robot.left_motor.value = left_value
    robot.right_motor.value = right_value
    
    # Telemetrie (Bez ukládání obrázků)
    frame_counter += 1
    if frame_counter % 20 == 0:
        print(f"FPS: {fps:.1f} | X: {x:.2f} | Motory: {left_value:.2f}/{right_value:.2f}")

Slepá (1.00) -> Manévr
Slepá (0.97) -> Manévr
Slepá (0.94) -> Manévr
Slepá (0.97) -> Manévr
Slepá (0.99) -> Manévr
Slepá (0.97) -> Manévr
Slepá (0.98) -> Manévr
Slepá (0.95) -> Manévr
FPS: 0.5 | X: 0.29 | Motory: 0.16/0.02
Slepá (0.94) -> Manévr


In [9]:
# Zavoláme funkci jednou ručně pro inicializaci (aby se načetly modely na GPU)
execute({'new': camera.value}) 

# Okamžitě zastavíme robota (kdyby náhodou model dal pokyn k jízdě)
robot.stop()

print("Inicializace hotova. Modely jsou připraveny ke startu.")

Inicializace hotova. Modely jsou připraveny ke startu.


In [23]:
# Připojíme funkci 'execute' ke kameře
# Tím se spustí smyčka řízení v reálném čase
camera.observe(execute, names='value')

FPS: 6.3 | X: 0.08 | Motory: 0.08/0.10
Slepá (0.94) -> Manévr
Slepá (0.90) -> Manévr
FPS: 0.5 | X: 0.07 | Motory: 0.08/0.10
FPS: 6.2 | X: -0.20 | Motory: 0.04/0.14
Slepá (0.93) -> Manévr
Slepá (0.99) -> Manévr
Slepá (1.00) -> Manévr
Slepá (0.89) -> Manévr
Slepá (0.89) -> Manévr
Slepá (0.95) -> Manévr
Slepá (0.98) -> Manévr
Slepá (0.94) -> Manévr
FPS: 6.3 | X: 0.17 | Motory: 0.12/0.06
Slepá (0.91) -> Manévr
FPS: 6.3 | X: -0.09 | Motory: 0.02/0.16
Slepá (0.95) -> Manévr
Slepá (0.99) -> Manévr
Slepá (0.99) -> Manévr
Slepá (0.85) -> Manévr
Slepá (0.94) -> Manévr
Slepá (1.00) -> Manévr


In [21]:
import time

# 1. Odpojíme kameru (přestane posílat nové snímky)
try:
    camera.unobserve(execute, names='value')
except:
    print("Kamera už byla odpojena.")

# 2. DŮLEŽITÉ: Počkáme chvilku, aby doběhla poslední funkce execute
time.sleep(0.2) 

# 3. Natvrdo vynulujeme motory (robot.stop() někdy selže, pokud je přetížený I2C)
robot.left_motor.value = 0.0
robot.right_motor.value = 0.0
robot.stop()

# 4. Resetujeme globální proměnné (aby robot při příštím startu "nebláznil")
angle = 0.0
angle_last = 0.0
is_recovering = False

print("🛑 Robot okamžitě zastaven a systém resetován.")

🛑 Robot okamžitě zastaven a systém resetován.


In [11]:
import glob
import cv2
import os

def create_ordered_img_array():
    img_name_array = []
    # ZMĚNA: Hledáme koncovku .jpg (shoda s funkcí save_frames)
    for filename in glob.glob('images/*.jpg'):
        img_name_array.append(filename)
    
    # Seřadíme podle názvu (jelikož názvy obsahují čas, seřadí se chronologicky)
    img_name_array.sort()

    img_array = []
    for filename in img_name_array:    
        img = cv2.imread(filename)
        if img is not None: # Kontrola, zda se obrázek načetl
            height, width, layers = img.shape
            size = (width,height)
            img_array.append(img)
            
    print(f'{len(img_array)} snímků nahráno pro video.')    
    return img_array   

def make_video(video_file_name, array_of_images, fps=15, image_size=(224,224)):
    if len(array_of_images) == 0:
        print("Varování: Žádné snímky pro vytvoření videa.")
        return

    # Používáme kodek DIVX (standard pro .avi)
    out = cv2.VideoWriter(video_file_name, cv2.VideoWriter_fourcc(*'DIVX'), fps, image_size)
    for i in range(len(array_of_images)):
        out.write(array_of_images[i])
    out.release()
    print(f"Video {video_file_name} uloženo.")
    
def delete_images():
    # ZMĚNA: Mažeme soubory .jpg
    count = 0
    if not os.path.exists('images'):
        return
        
    for image_file_name in os.listdir('images'):
        if image_file_name.endswith(".jpg"):
            os.remove('images/' + image_file_name)
            count += 1
    print(f"Smazáno {count} dočasných snímků.")

In [20]:
# Create array of images
img_array = create_ordered_img_array()

# Make video with 1 fps
make_video('video_1_fps.avi',img_array,1)

# Make video with 15 fps (actual speed)
make_video('video_15_fps.avi',img_array,15)

109 snímků nahráno pro video.
Video video_1_fps.avi uloženo.
Video video_15_fps.avi uloženo.


In [17]:
# Delete all images (Clean image folder)
delete_images()

Smazáno 158 dočasných snímků.


In [27]:
camera_link.unlink()  # don't stream to browser (will still run camera)

In [28]:
camera_link.link()  # stream to browser (wont run camera)